#### imports

In [1]:
import csv
import requests
import math
import pprint
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime, date, time

In [2]:
cwd = os.getcwd()
print cwd

/Users/jpy/Dropbox/_FLASK/__JUPYTER__/OCF


In [3]:
pp = pprint.PrettyPrinter(indent=2)

#### common functions

In [4]:
def findFromPathString(path_string, JSON, separator="/"):     
  paths = path_string.split(separator)
  data = JSON
  for i in range(0,len(paths)):
    data = data[paths[i]]
  return data

In [5]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

#### settings

In [6]:

root_url_corpos = "http://opencorporatefacts.fr/api/corporates"
# params_corpos = { 'page' : 1 , 'id' : None , 'CompanyNumber' : None }
results_path_corpos = "hydra:member"
output_filename_corpos = "corpos"

root_url_accounts = "http://opencorporatefacts.fr/api/compte_de_resultats"
# params_accounts = { 'page' : 1, 'Corporate' : None }
results_path_accounts = "hydra:member"
output_filename_accounts = "accounts"

root_url_companies = "http://api.cquest.org/company/" ### pass CompanyNumber after root_url
output_filename_companies = "companies"


#### tests

In [7]:
### test requests
resp = requests.get( root_url_corpos, params = { 'page' : 1 } ).json()
results = findFromPathString( results_path_corpos, resp )

In [8]:
print pp.pprint(results)

[ { u'@id': u'/api/corporates/2',
    u'@type': u'Corporate',
    u'CompanyNumber': u'414119735',
    u'ComptesDeResultats': [u'/api/compte_de_resultats/1'],
    u'CorporateShareholders': [],
    u'IndustryCode': u'',
    u'Name': u'AYMING',
    u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/414119735',
    u'documentDeReferences': [],
    u'id': 2,
    u'shareholdings': []},
  { u'@id': u'/api/corporates/3',
    u'@type': u'Corporate',
    u'CompanyNumber': u'419373055',
    u'ComptesDeResultats': [ u'/api/compte_de_resultats/7713',
                             u'/api/compte_de_resultats/2'],
    u'CorporateShareholders': [],
    u'IndustryCode': u'',
    u'Name': u'NORTH SOUTH CONNECTIONS FRANCE',
    u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/419373055',
    u'documentDeReferences': [],
    u'id': 3,
    u'shareholdings': []},
  { u'@id': u'/api/corporates/4',
    u'@type': u'Corporate',
    u'CompanyNumber': u'329936793',
    u'ComptesDeResultats

In [9]:
for i in results : 
    print i
    print

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/414119735', u'Name': u'AYMING', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/1'], u'documentDeReferences': [], u'IndustryCode': u'', u'id': 2, u'@id': u'/api/corporates/2', u'@type': u'Corporate', u'CompanyNumber': u'414119735'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/419373055', u'Name': u'NORTH SOUTH CONNECTIONS FRANCE', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/7713', u'/api/compte_de_resultats/2'], u'documentDeReferences': [], u'IndustryCode': u'', u'id': 3, u'@id': u'/api/corporates/3', u'@type': u'Corporate', u'CompanyNumber': u'419373055'}

{u'OpenCorporateURL': u'https://opencorporates.com/companies/fr/329936793', u'Name': u'MECAHERS GROUP', u'shareholdings': [], u'CorporateShareholders': [], u'ComptesDeResultats': [u'/api/compte_de_resultats/3'], u'documentDeReferences': [

#### main requests functions

In [16]:
def get_req_results ( url, pages_range, number_of_pages, results_path, output_filename, start_page=1, debug=False, out_formats=["csv"] ) : 
    ''' 
    retrieve results from requests 
    and write csv 
    works with API platform only for now
    '''
    
    start_time = datetime.now()
    
    ### output file
    if "csv" in out_formats :
        file_data_csv = cwd + '/' + output_filename + '.csv'
    if "json" in out_formats : 
        file_data_json = cwd + '/' + output_filename + '.json'
    
    ### count pages
    count = 0
    
    ### requests and writing file
    
    for page in pages_range : 
        
        if page > 0 and page >= start_page  : 
            
            print "\n page number : {} / {}".format(page, number_of_pages)
            
            resp = requests.get( url, params = { 'page' : page } ).json()
            results = findFromPathString( results_path, resp )
            
            page_time = datetime.now()
            delta_time = page_time - start_time
            print "delta_time (after request) : {} ".format(delta_time)

            # loop results
            for data in results : 
                
                if debug : 
                    # print pp.pprint(data) 
                    print data
                    print
                data_flat = flatten_json(data)
                # print data_flat
                
                ### CSV
                if "csv" in out_formats : 
                    
                    ### creates headers
                    if count == 0:
                        with open(file_data_csv, 'w') as csv_file :
                            csv_writer = csv.writer(csv_file, delimiter=';')
                            header = data.keys()
                            csv_writer.writerow(header)
                            count += 1
                    else : 
                        with open(file_data_csv, 'a') as csv_file :
                            csv_writer = csv.writer(csv_file, delimiter=';')
                            csv_writer.writerow( data.values() )
                            count += 1

                        
                ### JSON
                if "json" in out_formats : 
                    if count == 0 :
                        with open(file_data_json, 'w') as json_file :
                            json_file.write(json.dumps( [data] ).encode())         #If empty, write an array
                            count += 1
                    else : 
                        with open(file_data_json, 'ab+') as json_file :
                            json_file.seek(-1,2)           
                            json_file.truncate()                           #Remove the last character, open the array
                            json_file.write(' , '.encode())                #Write the separator
                            json_file.write(json.dumps(data).encode())     #Dump the dictionary
                            json_file.write(']'.encode())                  #Close the array
                            count += 1


#### getting data

In [17]:
### TEST / get corpos 
number_of_pages_accounts_test = 3
pages_range_test = range(number_of_pages_accounts_test)
get_req_results( 
    root_url_accounts, 
    pages_range_test, 
    number_of_pages_accounts_test,
    results_path_accounts, 
    output_filename_accounts, 
    debug=True, 
    out_formats=["json"] 
) 


 page number : 1 / 3
delta_time (after request) : 0:00:00.430416 
{u'DotationProvisions': 4600261, u'EffectifsMoyens': None, u'DifferenceNegativeChange': 69403, u'RepriseDepreciationEtProvisionTransfertsCharges': 2428200, u'ResultatFinancier': 2211444, u'DotationsFinancieresAmortissementDepreciationProvision': 4270802, u'ChiffresAffairesNet': 101067859, u'VariationStockMarchandise': 0, u'year': 2015, u'DotationExceptionnelleAmortissementDepreciationProvision': 25788996, u'RepriseDepreciationProvisionTransfertCharge': 0, u'AchatsDeMarchandises': 3153992, u'Corporate': u'/api/corporates/2', u'id': 1, u'VenteMarchandises': 0, u'ImpotTaxesEtVersementsAssimiles': 2934353, u'AutresAchatEtChargesExternes': 37905323, u'Benefice': -38665364, u'ResultatExploitation': -2687540, u'AutreInteretEtProduitAssimile': 36625, u'ParticipationSalariesAuxResultats': 293544, u'InteretEtChargeAssimilees': 66345, u'ProduitsExploitation': 116282174, u'ChargesFinancieres': 4406550, u'AutresCharges': 2108910, u'

delta_time (after request) : 0:00:00.803649 
{u'DotationProvisions': 0, u'EffectifsMoyens': -1, u'DifferenceNegativeChange': 0, u'RepriseDepreciationEtProvisionTransfertsCharges': 0, u'ResultatFinancier': -3164, u'DotationsFinancieresAmortissementDepreciationProvision': 0, u'ChiffresAffairesNet': 723494, u'VariationStockMarchandise': -60886, u'year': 2016, u'DotationExceptionnelleAmortissementDepreciationProvision': 0, u'RepriseDepreciationProvisionTransfertCharge': 0, u'AchatsDeMarchandises': 338287, u'Corporate': u'/api/corporates/98', u'id': 31, u'VenteMarchandises': 723277, u'ImpotTaxesEtVersementsAssimiles': 4090, u'AutresAchatEtChargesExternes': 192117, u'Benefice': 16224, u'ResultatExploitation': 20771, u'AutreInteretEtProduitAssimile': 38, u'ParticipationSalariesAuxResultats': 0, u'InteretEtChargeAssimilees': 3202, u'ProduitsExploitation': 731746, u'ChargesFinancieres': 3202, u'AutresCharges': 40, u'Dividende': -1, u'ChargesNetteCessionValeurMobiliereDePlacement': 0, u'Produits

In [ ]:
### get corpos data
number_of_results_corpos = 577297
results_per_page = 30
number_of_pages_corpos = int( math.ceil( number_of_results_corpos / results_per_page ) )
print number_of_pages_corpos
pages_to_scrap_corpos = range( number_of_pages_corpos )
print pages_to_scrap_corpos[:3]

get_req_results( 
    root_url_corpos, 
    pages_to_scrap_corpos, 
    number_of_pages_corpos,
    results_path_corpos, 
    output_filename_corpos+'_02', 
    debug=False, 
    out_formats=["json"] 
) 


 page number : 1 / 19243 


 page number : 2 / 19243 


 page number : 3 / 19243 


 page number : 4 / 19243 


 page number : 5 / 19243 


 page number : 6 / 19243 


 page number : 7 / 19243 


 page number : 8 / 19243 


 page number : 9 / 19243 


 page number : 10 / 19243 


 page number : 11 / 19243 


 page number : 12 / 19243 


 page number : 13 / 19243 


 page number : 14 / 19243 


 page number : 15 / 19243 


 page number : 16 / 19243 


 page number : 17 / 19243 


 page number : 18 / 19243 


 page number : 19 / 19243 


 page number : 20 / 19243 


 page number : 21 / 19243 


 page number : 22 / 19243 


 page number : 23 / 19243 


 page number : 24 / 19243 


 page number : 25 / 19243 


 page number : 26 / 19243 


 page number : 27 / 19243 


 page number : 28 / 19243 


 page number : 29 / 19243 


 page number : 30 / 19243 


 page number : 31 / 19243 


 page number : 32 / 19243 


 page number : 33 / 19243 


 page number : 34 / 19243 


 page number : 35 / 19


 page number : 278 / 19243 


 page number : 279 / 19243 


 page number : 280 / 19243 


 page number : 281 / 19243 


 page number : 282 / 19243 


 page number : 283 / 19243 


 page number : 284 / 19243 


 page number : 285 / 19243 


 page number : 286 / 19243 


 page number : 287 / 19243 


 page number : 288 / 19243 


 page number : 289 / 19243 


 page number : 290 / 19243 


 page number : 291 / 19243 


 page number : 292 / 19243 


 page number : 293 / 19243 


 page number : 294 / 19243 


 page number : 295 / 19243 


 page number : 296 / 19243 


 page number : 297 / 19243 


 page number : 298 / 19243 


 page number : 299 / 19243 


 page number : 300 / 19243 


 page number : 301 / 19243 


 page number : 302 / 19243 


 page number : 303 / 19243 


 page number : 304 / 19243 


 page number : 305 / 19243 


 page number : 306 / 19243 


 page number : 307 / 19243 


 page number : 308 / 19243 


 page number : 309 / 19243 


 page number : 310 / 19243 


 page num


 page number : 552 / 19243 


 page number : 553 / 19243 


 page number : 554 / 19243 


 page number : 555 / 19243 


 page number : 556 / 19243 


 page number : 557 / 19243 


 page number : 558 / 19243 


 page number : 559 / 19243 


 page number : 560 / 19243 


 page number : 561 / 19243 


 page number : 562 / 19243 


 page number : 563 / 19243 


 page number : 564 / 19243 


 page number : 565 / 19243 


 page number : 566 / 19243 


 page number : 567 / 19243 


 page number : 568 / 19243 


 page number : 569 / 19243 


 page number : 570 / 19243 


 page number : 571 / 19243 


 page number : 572 / 19243 


 page number : 573 / 19243 


 page number : 574 / 19243 


 page number : 575 / 19243 


 page number : 576 / 19243 


 page number : 577 / 19243 


 page number : 578 / 19243 


 page number : 579 / 19243 


 page number : 580 / 19243 


 page number : 581 / 19243 


 page number : 582 / 19243 


 page number : 583 / 19243 


 page number : 584 / 19243 


 page num


 page number : 826 / 19243 


 page number : 827 / 19243 


 page number : 828 / 19243 


 page number : 829 / 19243 


 page number : 830 / 19243 


 page number : 831 / 19243 


 page number : 832 / 19243 


 page number : 833 / 19243 


 page number : 834 / 19243 


 page number : 835 / 19243 


 page number : 836 / 19243 


 page number : 837 / 19243 


 page number : 838 / 19243 


 page number : 839 / 19243 


 page number : 840 / 19243 


 page number : 841 / 19243 


 page number : 842 / 19243 


 page number : 843 / 19243 


 page number : 844 / 19243 


 page number : 845 / 19243 


 page number : 846 / 19243 


 page number : 847 / 19243 


 page number : 848 / 19243 


 page number : 849 / 19243 


 page number : 850 / 19243 


 page number : 851 / 19243 


 page number : 852 / 19243 


 page number : 853 / 19243 


 page number : 854 / 19243 


 page number : 855 / 19243 


 page number : 856 / 19243 


 page number : 857 / 19243 


 page number : 858 / 19243 


 page num


 page number : 1096 / 19243 


 page number : 1097 / 19243 


 page number : 1098 / 19243 


 page number : 1099 / 19243 


 page number : 1100 / 19243 


 page number : 1101 / 19243 


 page number : 1102 / 19243 


 page number : 1103 / 19243 


 page number : 1104 / 19243 


 page number : 1105 / 19243 


 page number : 1106 / 19243 


 page number : 1107 / 19243 


 page number : 1108 / 19243 


 page number : 1109 / 19243 


 page number : 1110 / 19243 


 page number : 1111 / 19243 


 page number : 1112 / 19243 


 page number : 1113 / 19243 


 page number : 1114 / 19243 


 page number : 1115 / 19243 


 page number : 1116 / 19243 


 page number : 1117 / 19243 


 page number : 1118 / 19243 


 page number : 1119 / 19243 


 page number : 1120 / 19243 


 page number : 1121 / 19243 


 page number : 1122 / 19243 


 page number : 1123 / 19243 


 page number : 1124 / 19243 


 page number : 1125 / 19243 


 page number : 1126 / 19243 


 page number : 1127 / 19243 


 page n


 page number : 1361 / 19243 


 page number : 1362 / 19243 


 page number : 1363 / 19243 


 page number : 1364 / 19243 


 page number : 1365 / 19243 


 page number : 1366 / 19243 


 page number : 1367 / 19243 


 page number : 1368 / 19243 


 page number : 1369 / 19243 


 page number : 1370 / 19243 


 page number : 1371 / 19243 


 page number : 1372 / 19243 


 page number : 1373 / 19243 


 page number : 1374 / 19243 


 page number : 1375 / 19243 


 page number : 1376 / 19243 


 page number : 1377 / 19243 


 page number : 1378 / 19243 


 page number : 1379 / 19243 


 page number : 1380 / 19243 


 page number : 1381 / 19243 


 page number : 1382 / 19243 


 page number : 1383 / 19243 


 page number : 1384 / 19243 


 page number : 1385 / 19243 


 page number : 1386 / 19243 


 page number : 1387 / 19243 


 page number : 1388 / 19243 


 page number : 1389 / 19243 


 page number : 1390 / 19243 


 page number : 1391 / 19243 


 page number : 1392 / 19243 


 page n


 page number : 1626 / 19243 


 page number : 1627 / 19243 


 page number : 1628 / 19243 


 page number : 1629 / 19243 


 page number : 1630 / 19243 


 page number : 1631 / 19243 


 page number : 1632 / 19243 


 page number : 1633 / 19243 


 page number : 1634 / 19243 


 page number : 1635 / 19243 


 page number : 1636 / 19243 


 page number : 1637 / 19243 


 page number : 1638 / 19243 


 page number : 1639 / 19243 


 page number : 1640 / 19243 


 page number : 1641 / 19243 


 page number : 1642 / 19243 


 page number : 1643 / 19243 


 page number : 1644 / 19243 


 page number : 1645 / 19243 


 page number : 1646 / 19243 


 page number : 1647 / 19243 


 page number : 1648 / 19243 


 page number : 1649 / 19243 


 page number : 1650 / 19243 


 page number : 1651 / 19243 


 page number : 1652 / 19243 


 page number : 1653 / 19243 


 page number : 1654 / 19243 


 page number : 1655 / 19243 


 page number : 1656 / 19243 


 page number : 1657 / 19243 


 page n


 page number : 1891 / 19243 


 page number : 1892 / 19243 


 page number : 1893 / 19243 


 page number : 1894 / 19243 


 page number : 1895 / 19243 


 page number : 1896 / 19243 


 page number : 1897 / 19243 


 page number : 1898 / 19243 


 page number : 1899 / 19243 


 page number : 1900 / 19243 


 page number : 1901 / 19243 


 page number : 1902 / 19243 


 page number : 1903 / 19243 


 page number : 1904 / 19243 


 page number : 1905 / 19243 


 page number : 1906 / 19243 


 page number : 1907 / 19243 


 page number : 1908 / 19243 


 page number : 1909 / 19243 


 page number : 1910 / 19243 


 page number : 1911 / 19243 


 page number : 1912 / 19243 


 page number : 1913 / 19243 


 page number : 1914 / 19243 


 page number : 1915 / 19243 


 page number : 1916 / 19243 


 page number : 1917 / 19243 


 page number : 1918 / 19243 


 page number : 1919 / 19243 


 page number : 1920 / 19243 


 page number : 1921 / 19243 


 page number : 1922 / 19243 


 page n


 page number : 2156 / 19243 


 page number : 2157 / 19243 


 page number : 2158 / 19243 


 page number : 2159 / 19243 


 page number : 2160 / 19243 


 page number : 2161 / 19243 


 page number : 2162 / 19243 


 page number : 2163 / 19243 


 page number : 2164 / 19243 


 page number : 2165 / 19243 


 page number : 2166 / 19243 


 page number : 2167 / 19243 


 page number : 2168 / 19243 


 page number : 2169 / 19243 


 page number : 2170 / 19243 


 page number : 2171 / 19243 


 page number : 2172 / 19243 


 page number : 2173 / 19243 


 page number : 2174 / 19243 


 page number : 2175 / 19243 


 page number : 2176 / 19243 


 page number : 2177 / 19243 


 page number : 2178 / 19243 


 page number : 2179 / 19243 


 page number : 2180 / 19243 


 page number : 2181 / 19243 


 page number : 2182 / 19243 


 page number : 2183 / 19243 


 page number : 2184 / 19243 


 page number : 2185 / 19243 


 page number : 2186 / 19243 


 page number : 2187 / 19243 


 page n


 page number : 2421 / 19243 


 page number : 2422 / 19243 


 page number : 2423 / 19243 


 page number : 2424 / 19243 


 page number : 2425 / 19243 


 page number : 2426 / 19243 


 page number : 2427 / 19243 


 page number : 2428 / 19243 


 page number : 2429 / 19243 


 page number : 2430 / 19243 


 page number : 2431 / 19243 


 page number : 2432 / 19243 


 page number : 2433 / 19243 


 page number : 2434 / 19243 


 page number : 2435 / 19243 


 page number : 2436 / 19243 


 page number : 2437 / 19243 


 page number : 2438 / 19243 


 page number : 2439 / 19243 


 page number : 2440 / 19243 


 page number : 2441 / 19243 


 page number : 2442 / 19243 


 page number : 2443 / 19243 


 page number : 2444 / 19243 


 page number : 2445 / 19243 


 page number : 2446 / 19243 


 page number : 2447 / 19243 


 page number : 2448 / 19243 


 page number : 2449 / 19243 


 page number : 2450 / 19243 


 page number : 2451 / 19243 


 page number : 2452 / 19243 


 page n

In [ ]:
### get accounts data
number_of_results_account = 316182
results_per_page = 30
number_of_pages_accounts = int( math.ceil( number_of_results_account / results_per_page ) )
print number_of_pages_accounts
pages_to_scrap_accounts = range( number_of_pages_accounts )
print pages_to_scrap_accounts[:3]

get_req_results( 
    root_url_accounts, 
    pages_to_scrap_accounts, 
    number_of_pages_accounts,
    results_path_accounts, 
    output_filename_accounts+'_01', 
    debug=False, 
    out_formats=["json"] 
) 

10539
[0, 1, 2]

 page number : 1 / 10539
delta_time (after request) : 0:00:00.398655 

 page number : 2 / 10539
delta_time (after request) : 0:00:00.892737 

 page number : 3 / 10539
delta_time (after request) : 0:00:01.155158 

 page number : 4 / 10539
delta_time (after request) : 0:00:01.675778 

 page number : 5 / 10539
delta_time (after request) : 0:00:01.987405 

 page number : 6 / 10539
delta_time (after request) : 0:00:02.394938 

 page number : 7 / 10539
delta_time (after request) : 0:00:02.753324 

 page number : 8 / 10539
delta_time (after request) : 0:00:03.332381 

 page number : 9 / 10539
delta_time (after request) : 0:00:03.655075 

 page number : 10 / 10539
delta_time (after request) : 0:00:04.239913 

 page number : 11 / 10539
delta_time (after request) : 0:00:04.597157 

 page number : 12 / 10539
delta_time (after request) : 0:00:05.051489 

 page number : 13 / 10539
delta_time (after request) : 0:00:05.333025 

 page number : 14 / 10539
delta_time (after request) : 0

delta_time (after request) : 0:00:54.047052 

 page number : 116 / 10539
delta_time (after request) : 0:00:54.529613 

 page number : 117 / 10539
delta_time (after request) : 0:00:54.923731 

 page number : 118 / 10539
delta_time (after request) : 0:00:55.426250 

 page number : 119 / 10539
delta_time (after request) : 0:00:55.853866 

 page number : 120 / 10539
delta_time (after request) : 0:00:56.495545 

 page number : 121 / 10539
delta_time (after request) : 0:00:56.898090 

 page number : 122 / 10539
delta_time (after request) : 0:00:57.819301 

 page number : 123 / 10539
delta_time (after request) : 0:00:58.537311 

 page number : 124 / 10539
delta_time (after request) : 0:00:59.508692 

 page number : 125 / 10539
delta_time (after request) : 0:01:00.297111 

 page number : 126 / 10539
delta_time (after request) : 0:01:00.991823 

 page number : 127 / 10539
delta_time (after request) : 0:01:01.556089 

 page number : 128 / 10539
delta_time (after request) : 0:01:02.149422 

 page

delta_time (after request) : 0:01:55.958970 

 page number : 229 / 10539
delta_time (after request) : 0:01:56.353559 

 page number : 230 / 10539
delta_time (after request) : 0:01:56.936102 

 page number : 231 / 10539
delta_time (after request) : 0:01:57.817548 

 page number : 232 / 10539
delta_time (after request) : 0:01:58.823203 

 page number : 233 / 10539
delta_time (after request) : 0:01:59.670802 

 page number : 234 / 10539
delta_time (after request) : 0:02:00.051574 

 page number : 235 / 10539
delta_time (after request) : 0:02:00.796438 

 page number : 236 / 10539
delta_time (after request) : 0:02:01.297778 

 page number : 237 / 10539
delta_time (after request) : 0:02:01.741412 

 page number : 238 / 10539
delta_time (after request) : 0:02:02.218354 

 page number : 239 / 10539
delta_time (after request) : 0:02:02.652031 

 page number : 240 / 10539
delta_time (after request) : 0:02:03.274245 

 page number : 241 / 10539
delta_time (after request) : 0:02:03.942771 

 page

delta_time (after request) : 0:03:04.608914 

 page number : 342 / 10539
delta_time (after request) : 0:03:05.183740 

 page number : 343 / 10539
delta_time (after request) : 0:03:05.813554 

 page number : 344 / 10539
delta_time (after request) : 0:03:06.557089 

 page number : 345 / 10539
delta_time (after request) : 0:03:07.350221 

 page number : 346 / 10539
delta_time (after request) : 0:03:08.172086 

 page number : 347 / 10539
delta_time (after request) : 0:03:08.634315 

 page number : 348 / 10539
delta_time (after request) : 0:03:09.073507 

 page number : 349 / 10539
delta_time (after request) : 0:03:09.603228 

 page number : 350 / 10539
delta_time (after request) : 0:03:10.462858 

 page number : 351 / 10539
delta_time (after request) : 0:03:11.119477 

 page number : 352 / 10539
delta_time (after request) : 0:03:11.729850 

 page number : 353 / 10539
delta_time (after request) : 0:03:12.299716 

 page number : 354 / 10539
delta_time (after request) : 0:03:12.747079 

 page

delta_time (after request) : 0:04:24.792919 

 page number : 455 / 10539
delta_time (after request) : 0:04:25.442352 

 page number : 456 / 10539
delta_time (after request) : 0:04:26.675250 

 page number : 457 / 10539
delta_time (after request) : 0:04:27.395345 

 page number : 458 / 10539
delta_time (after request) : 0:04:28.027737 

 page number : 459 / 10539
delta_time (after request) : 0:04:28.621676 

 page number : 460 / 10539
delta_time (after request) : 0:04:29.557599 

 page number : 461 / 10539
delta_time (after request) : 0:04:30.276065 

 page number : 462 / 10539
delta_time (after request) : 0:04:30.948687 

 page number : 463 / 10539
delta_time (after request) : 0:04:31.424484 

 page number : 464 / 10539
delta_time (after request) : 0:04:32.029233 

 page number : 465 / 10539
delta_time (after request) : 0:04:32.872227 

 page number : 466 / 10539
delta_time (after request) : 0:04:33.764925 

 page number : 467 / 10539
delta_time (after request) : 0:04:34.413481 

 page

...

delta_time (after request) : 0:14:37.546060 

 page number : 1017 / 10539
delta_time (after request) : 0:14:38.288683 

 page number : 1018 / 10539
delta_time (after request) : 0:14:39.087264 

 page number : 1019 / 10539
delta_time (after request) : 0:14:40.093061 

 page number : 1020 / 10539
delta_time (after request) : 0:14:40.885871 

 page number : 1021 / 10539
delta_time (after request) : 0:14:41.689543 

 page number : 1022 / 10539
delta_time (after request) : 0:14:42.726538 

 page number : 1023 / 10539
delta_time (after request) : 0:14:43.876542 

 page number : 1024 / 10539
delta_time (after request) : 0:14:44.902339 

 page number : 1025 / 10539
delta_time (after request) : 0:14:45.956967 

 page number : 1026 / 10539
delta_time (after request) : 0:14:46.820366 

 page number : 1027 / 10539
delta_time (after request) : 0:14:47.621720 

 page number : 1028 / 10539
delta_time (after request) : 0:14:48.868558 

 page number : 1029 / 10539
delta_time (after request) : 0:14:50.1

delta_time (after request) : 0:17:40.748686 

 page number : 1128 / 10539
delta_time (after request) : 0:17:42.090041 

 page number : 1129 / 10539
delta_time (after request) : 0:17:43.314445 

 page number : 1130 / 10539
delta_time (after request) : 0:17:44.739617 

 page number : 1131 / 10539
delta_time (after request) : 0:17:46.483347 

 page number : 1132 / 10539
delta_time (after request) : 0:17:47.814121 

 page number : 1133 / 10539
delta_time (after request) : 0:17:50.131043 

 page number : 1134 / 10539
delta_time (after request) : 0:17:53.327595 

 page number : 1135 / 10539
delta_time (after request) : 0:17:55.222607 

 page number : 1136 / 10539
delta_time (after request) : 0:17:58.749699 

 page number : 1137 / 10539
delta_time (after request) : 0:18:00.204906 

 page number : 1138 / 10539
delta_time (after request) : 0:18:01.934595 

 page number : 1139 / 10539
delta_time (after request) : 0:18:03.797487 

 page number : 1140 / 10539
delta_time (after request) : 0:18:05.7

delta_time (after request) : 0:21:15.959741 

 page number : 1239 / 10539
delta_time (after request) : 0:21:17.133978 

 page number : 1240 / 10539
delta_time (after request) : 0:21:18.906823 

 page number : 1241 / 10539
delta_time (after request) : 0:21:20.963730 

 page number : 1242 / 10539
delta_time (after request) : 0:21:22.816368 

 page number : 1243 / 10539
delta_time (after request) : 0:21:24.670606 

 page number : 1244 / 10539
delta_time (after request) : 0:21:26.435305 

 page number : 1245 / 10539
delta_time (after request) : 0:21:28.276378 

 page number : 1246 / 10539
delta_time (after request) : 0:21:29.955693 

 page number : 1247 / 10539
delta_time (after request) : 0:21:31.784831 

 page number : 1248 / 10539
delta_time (after request) : 0:21:36.139323 

 page number : 1249 / 10539
delta_time (after request) : 0:21:37.181915 

 page number : 1250 / 10539
delta_time (after request) : 0:21:38.928279 

 page number : 1251 / 10539
delta_time (after request) : 0:21:42.5

delta_time (after request) : 0:24:19.767275 

 page number : 1350 / 10539
delta_time (after request) : 0:24:20.891868 

 page number : 1351 / 10539
delta_time (after request) : 0:24:21.922638 

 page number : 1352 / 10539
delta_time (after request) : 0:24:23.406944 

 page number : 1353 / 10539
delta_time (after request) : 0:24:24.574309 

 page number : 1354 / 10539
delta_time (after request) : 0:24:25.569620 

 page number : 1355 / 10539
delta_time (after request) : 0:24:26.489484 

 page number : 1356 / 10539
delta_time (after request) : 0:24:27.333035 

 page number : 1357 / 10539
delta_time (after request) : 0:24:28.224806 

 page number : 1358 / 10539
delta_time (after request) : 0:24:29.115073 

 page number : 1359 / 10539
delta_time (after request) : 0:24:29.920782 

 page number : 1360 / 10539
delta_time (after request) : 0:24:31.029299 

 page number : 1361 / 10539
delta_time (after request) : 0:24:32.088848 

 page number : 1362 / 10539
delta_time (after request) : 0:24:33.3

delta_time (after request) : 0:27:02.694370 

 page number : 1461 / 10539
delta_time (after request) : 0:27:04.052265 

 page number : 1462 / 10539
delta_time (after request) : 0:27:05.397524 

 page number : 1463 / 10539
delta_time (after request) : 0:27:07.468965 

 page number : 1464 / 10539
delta_time (after request) : 0:27:08.673279 

 page number : 1465 / 10539
delta_time (after request) : 0:27:10.270073 

 page number : 1466 / 10539
delta_time (after request) : 0:27:12.415116 

 page number : 1467 / 10539
delta_time (after request) : 0:27:14.155605 

 page number : 1468 / 10539
delta_time (after request) : 0:27:15.806402 

 page number : 1469 / 10539
delta_time (after request) : 0:27:21.137152 

 page number : 1470 / 10539
delta_time (after request) : 0:27:25.228237 

 page number : 1471 / 10539
delta_time (after request) : 0:27:26.130639 

 page number : 1472 / 10539
delta_time (after request) : 0:27:27.135068 

 page number : 1473 / 10539
delta_time (after request) : 0:27:28.3